In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow optuna


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import optuna
import joblib


In [ ]:
# Importar datos preprocesados desde el cuaderno_A
X_train_scaled_imputed = np.load('X_train.npy')
X_test_scaled_imputed = np.load('X_test.npy')
y_train = np.load('y_train.npy')



In [ ]:
# Modelo de Deep Learning
model_dl = Sequential()
model_dl.add(Dense(30, activation='relu', input_shape=(X_train_scaled_imputed.shape[1],)))
model_dl.add(Dense(1))
model_dl.compile(optimizer='adam', loss='mean_squared_error')

model_dl.fit(X_train_scaled_imputed, y_train, epochs=50, batch_size=32, validation_split=0.2)



In [ ]:
# Modelo de Machine Learning
model_ml = RandomForestRegressor(n_estimators=100, random_state=42)
model_ml.fit(X_train_scaled_imputed, y_train)


In [ ]:
# Evaluación del modelo de Deep Learning
y_test = test_data["median_house_value"].copy()
X_test_scaled_imputed = scaler.transform(X_test_imputed)
y_pred_dl_imputed = model_dl.predict(X_test_scaled_imputed)
mse_dl_imputed = mean_squared_error(y_test, y_pred_dl_imputed)
r2_dl_imputed = r2_score(y_test, y_pred_dl_imputed)
print(f"Deep Learning con Imputación - Mean Squared Error: {mse_dl_imputed}, R^2 Score: {r2_dl_imputed}")


In [ ]:
# Evaluación del modelo de Machine Learning
y_pred_ml_imputed = model_ml.predict(X_test_scaled_imputed)
mse_ml_imputed = mean_squared_error(y_test, y_pred_ml_imputed)
r2_ml_imputed = r2_score(y_test, y_pred_ml_imputed)
print(f"Machine Learning con Imputación - Mean Squared Error: {mse_ml_imputed}, R^2 Score: {r2_ml_imputed}")


In [ ]:
# Ajuste de Hiperparámetros (utilizando Optuna)
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train_scaled_imputed, y_train)
    y_pred = model.predict(X_test_scaled_imputed)
    return mean_squared_error(y_test, y_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
best_model = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                    max_depth=best_params['max_depth'], random_state=42)
best_model.fit(X_train_scaled_imputed, y_train)

y_pred_best = best_model.predict(X_test_scaled_imputed)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)
print(f"Best Model - Mean Squared Error: {mse_best}, R^2 Score: {r2_best}")

In [ ]:
# Ruta donde se guardarán los modelos
ruta_modelos = 'modelos/'

# Guardar el modelo de Deep Learning
model_dl.save(f'{ruta_modelos}modelo_dl')

# Guardar el modelo de Machine Learning
ruta_modelo_ml = f'{ruta_modelos}modelo_ml.joblib'
joblib.dump(model_ml, ruta_modelo_ml)

# Guardar el mejor modelo
ruta_mejor_modelo = f'{ruta_modelos}mejor_modelo.joblib'
joblib.dump(best_model, ruta_mejor_modelo)

# Puedes imprimir un mensaje indicando la ubicación de los modelos guardados
print(f"Modelos guardados en: {ruta_modelos}")